# DocumentSummaryIndex

## 准备

In [1]:
%%time

from llama_index.llms.openai_like import OpenAILike
from llama_index.core import Settings

llm = OpenAILike(model="qwen2-7b-6k", 
                 api_base="http://192.168.0.72:3000/v1", 
                 api_key="sk-bJP6QSnUfjAYeYeE505d3eBf63A643BeB0B8E350Df9b7750",
                 is_chat_model=True,
                 temperature=0.1,
                 request_timeout=60.0
                )

Settings.llm =llm

CPU times: user 2.68 s, sys: 405 ms, total: 3.08 s
Wall time: 2.73 s


In [2]:
%%time

# embedding
from llama_index.embeddings.ollama import OllamaEmbedding

ollama_embedding = OllamaEmbedding(
    # model_name="dztech/bge-large-zh:v1.5",
    # model_name="bge-m3:latest",
    model_name="chatfire/bge-m3:q8_0",
    base_url="http://192.168.0.72:11435",
    ollama_additional_kwargs={"mirostat": 0}, # -mirostat N 使用 Mirostat 采样。
)

Settings.embed_model = ollama_embedding

CPU times: user 522 ms, sys: 24.4 ms, total: 546 ms
Wall time: 546 ms


In [3]:
from llama_index.core import SimpleDirectoryReader

docs = SimpleDirectoryReader(
        input_files=[f"../data/news-14k.txt"]
    ).load_data()

docs[0].doc_id = "长新闻"

docs[0].text[:100]

'欧盟对华急征电车关税，一场“汽车大战”的预言即将兑现？｜文化纵横\n\n文化纵横《文化纵横》杂志官方账号昨天 10:02\n【导读】近日，欧盟委员会公布对中国电动汽车的反补贴调查结果，称中国的纯电动汽车价值'

In [4]:
len(docs[0].text)

13869

In [5]:
%%time

from llama_index.core.node_parser import SentenceSplitter
splitter = SentenceSplitter(chunk_size=1024)

CPU times: user 225 ms, sys: 24.2 ms, total: 250 ms
Wall time: 249 ms


## 创建索引

In [6]:
%%time

from llama_index.core import get_response_synthesizer
from llama_index.core import DocumentSummaryIndex

import nest_asyncio
nest_asyncio.apply()

response_synthesizer = get_response_synthesizer(
    response_mode="tree_summarize", use_async=True,
)

doc_summary_index = DocumentSummaryIndex.from_documents(
    [docs[0]],
    transformations=[splitter],
    response_synthesizer=response_synthesizer,
    show_progress=True,
)

Parsing nodes:   0%|          | 0/1 [00:00<?, ?it/s]

Summarizing documents:   0%|          | 0/1 [00:00<?, ?it/s]

current doc id: 长新闻


Generating embeddings:   0%|          | 0/1 [00:00<?, ?it/s]

CPU times: user 211 ms, sys: 8.14 ms, total: 219 ms
Wall time: 55.3 s


In [7]:
doc_summary_index.get_document_summary("长新闻")

'The provided text discusses the global competition in the field of electric vehicles (EVs), with a particular emphasis on China\'s rapid growth and its impact on traditional automotive powerhouses like Europe and America. The article highlights how China has become the world leader in both sales and production of EVs, with significant market share and export volumes.\n\nThe text also delves into why Western countries are taking aggressive measures against Chinese electric vehicle manufacturers, including imposing tariffs and restrictions on imports, as part of a broader strategy to maintain global competitiveness in the automotive industry amidst environmental concerns and technological advancements. \n\nSome key questions that this text can answer include:\n\n1. How has China\'s electric vehicle market grown over time?\n2. What role do electric vehicles play in shaping international competition between China and Western countries?\n3. Why are Western governments taking protective mea

## 查询

### 高级查询

In [8]:
%%time

query_engine = doc_summary_index.as_query_engine(
    response_mode="tree_summarize", 
    streaming=True,
    # similarity_top_k=5
)

CPU times: user 30.3 ms, sys: 3.33 ms, total: 33.7 ms
Wall time: 33 ms


In [9]:
%%time

response = query_engine.query("面对全球保护主义，中国电动汽车出口方式有何变化？")
response.print_response_stream()

面对全球保护主义的影响，中国电动汽车采取了以下策略调整其出口方式：

1. **市场多元化**：积极开拓新兴市场，如南亚、东南亚和中东地区。这些市场的汽车保有量相对较低，人口基数大，为电动汽车提供了增长潜力。

2. **直接投资与合作**：通过投资、并购等方式加强在目标市场的直接参与，以获得当地市场准入，并加速技术转移和产业整合。

3. **建立检测认证联盟**：与中国政府及部分发展中国家合作，建立检测认证联盟来简化出口流程并降低风险。这有助于应对不同国家的法规要求和标准差异。

4. **推动“新型全球化”**：探索不同于传统跨国公司模式的新路径，强调在价值创造与分配上的公平性和包容性，通过将南方国家纳入与中国相关的价值链中，促进其工业化进程。

5. **加强本土创新**：强化立足于本土的创新活动，形成基于中国技术议程和市场需求的核心竞争力。这些创新有助于辐射更广泛的国际范围，并确保在全球输出技术、产业和标准时保持竞争优势。

6. **关注关键金属资源安全**：鉴于锂、镍、钴等关键金属原材料供应与生产制造分离的情况，加强与中国掌握这些资源的矿产大国的合作关系，以保障供应链的安全并促进全球南方国家经济自主意识的提升和工业化进程。CPU times: user 418 ms, sys: 40.6 ms, total: 459 ms
Wall time: 1min 31s


### 基于 LLM 的召回

In [10]:
%%time

from llama_index.core.indices.document_summary import (
    DocumentSummaryIndexLLMRetriever,
)

CPU times: user 5 µs, sys: 1e+03 ns, total: 6 µs
Wall time: 6.44 µs


In [11]:
%%time

retriever = DocumentSummaryIndexLLMRetriever(
    doc_summary_index,
    # choice_select_prompt=None,
    # choice_batch_size=10,
    # choice_top_k=1,
    # format_node_batch_fn=None,
    # parse_choice_select_answer_fn=None,
)

retrieved_nodes = retriever.retrieve("面对全球保护主义，中国电动汽车出口方式有何变化？")

CPU times: user 10 ms, sys: 0 ns, total: 10 ms
Wall time: 4.81 s


In [12]:
len(retrieved_nodes)

19

In [13]:
retrieved_nodes[0].score

8.0

In [14]:
retrieved_nodes[0].node.get_text()

'欧盟对华急征电车关税，一场“汽车大战”的预言即将兑现？｜文化纵横\n\n文化纵横《文化纵横》杂志官方账号昨天 10:02\n【导读】近日，欧盟委员会公布对中国电动汽车的反补贴调查结果，称中国的纯电动汽车价值链受益于不公平补贴，损害了欧盟生产商的利益，并宣布将对中国生产的电动汽车征收最高38.1%的关税，最迟于7月4日正式生效。欧盟突然挥起贸易保护主义的大棒，给正在崛起的中国车企蒙上了一层阴影。6月13、14日，国家发改委在公众号连续发文，批评欧盟调查“罔顾事实、预设结果、不合理、不合规，本质是将调查工具武器化、政治化”，“对中国电动汽车加征关税只能暂时‘守住’欧盟本土市场，但在新能源转型大背景下，欧盟汽车产业的全球竞争力恐将进一步下降”。\n\n汽车工业对诸多不同的产业部门具有重要的拉动效应，其影响极有可能蔓延至国际竞争的诸多领域，成为新世纪中“改变世界的新机器”。这也是为什么西方国家会采取激烈的手段阻碍中国新能源汽车的市场扩张。本文是对中国新能源汽车产业崛起的一次深刻的总结与分析。\n\n文章指出，当前西方国家对中国新能源汽车采取的保护性产业政策，有重演20世纪后半期美日“汽车大战”的趋势。面对这一挑战，我国要探索汽车行业的“新型全球化”道路，积极开拓发展中国家的新兴市场，将南方国家的工业化活动纳入与中国相关的价值链，实质性地推动南方国家的工业化进程，充分提高发展中国家民众收入，并将其转变为新能源汽车的消费者。\n\n本文原载《文化纵横》2024年第2期，原题为《改变世界的新机器？——新能源汽车全球竞争的前景》，仅代表作者观点，供读者参考。\n\n改变世界的新机器？\n\n——新能源汽车全球竞争的前景\n\n封凯栋 陈俊廷\n\n北京大学政府管理学院\n\n公共治理研究所\n\n国家治理研究院\n\n在国际环境日益复杂，及环保、能源安全等议题越来越受关注的背景下，新能源汽车尤其是中国新能源汽车产业的发展成为世界瞩目的焦点。2016年以来，我国新能源汽车年销量和保有量持续居于世界第一。2023年，中国新能源汽车呈现爆发式增长，产销量分别达到958.7万辆和949.5万辆，销量占全球总销量的比例达到66%。'

In [15]:
%%time

from llama_index.core.query_engine import RetrieverQueryEngine

# configure response synthesizer
response_synthesizer = get_response_synthesizer(
    response_mode="tree_summarize",
    streaming=True
)

# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
)

# query
response = query_engine.query("面对全球保护主义，中国电动汽车出口方式有何变化？")
response.print_response_stream()

面对全球保护主义的趋势，中国电动汽车的出口方式发生了一些积极的变化。首先，在2023年，中国新能源汽车市场呈现爆发式增长，产销量分别达到958.7万辆和949.5万辆，其中新能源汽车的出口量达到了173万辆，占总出口量的比例超过30%。这表明中国电动汽车出口数量显著增加。

其次，中国电动汽车出口的质量也在不断提升。随着技术进步和市场竞争力增强，出口单价持续上升，显示了中国电动汽车在全球市场的竞争力提升。同时，中国电动汽车出口的地理分布也发生了变化，近一半销往德国、法国、英国、比利时等欧洲国家，这表明中国电动汽车成功地开拓了包括发达国家在内的国际市场。

这些变化反映了中国在面对全球保护主义时采取的战略调整，即积极开拓发展中国家和发达国家的新兴市场，通过提升产品质量和技术水平来增强国际竞争力。同时，中国也在探索汽车行业的“新型全球化”道路，将南方国家的工业化活动纳入与中国相关的价值链中，推动南方国家的工业化进程，并提高发展中国家民众收入，从而为新能源汽车消费创造更多潜在市场。

这些策略不仅有助于减少对中国电动汽车出口的贸易壁垒影响，还促进了全球新能源汽车产业的发展和合作。面对全球保护主义的挑战，中国电动汽车企业采取了灵活多变的策略来调整其出口方式：

1. **直接投资本地市场**：为了规避贸易壁垒和关税风险，并快速进入当地市场，中国电动汽车企业选择在欧美等国家进行直接投资，设立生产基地或研发中心。

2. **通过投资与并购获得技术资产**：跨国公司倾向于通过投资、并购等方式获取中国的新能源汽车技术资产。这加速了其自身的技术升级和转型进程，同时利用在中国积累的新能源技术优势，在全球范围内快速布局新能源市场。

3. **开拓新兴市场**：中国电动汽车企业积极开发此前未充分开发的新兴市场，如南亚、东南亚、中东等地区成为新的增长点。数据显示，2022年向泰国、菲律宾、印度、孟加拉等国家出口新能源汽车的数量显著增加，其中对泰国的出口量接近8万辆。

4. **利用政策优势**：部分南亚和东南亚国家实施了针对新能源汽车进口的税收优惠、购车补贴、消费税减免以及外商投资补贴等政策。这些优惠政策为中国的电动汽车产品和技术出口提供了有利条件。

5. **建立检测认证联盟**：中国与一些发展中国家政府合作，建立了新能源汽车出口过程中的检测认证联盟，简化和加速了产品的海外认证流程

### 基于嵌入的召回

In [16]:
%%time

from llama_index.core.indices.document_summary import (
    DocumentSummaryIndexEmbeddingRetriever,
)

retriever = DocumentSummaryIndexEmbeddingRetriever(
    doc_summary_index,
    # similarity_top_k=5,
)

retrieved_nodes = retriever.retrieve("面对全球保护主义，中国电动汽车出口方式有何变化？")

len(retrieved_nodes)

CPU times: user 3.31 ms, sys: 3.68 ms, total: 6.99 ms
Wall time: 1.8 s


19

In [17]:
%%time

# use retriever as part of a query engine
from llama_index.core.query_engine import RetrieverQueryEngine

# configure response synthesizer
response_synthesizer = get_response_synthesizer(
    response_mode="tree_summarize",
    streaming=True
)

# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
)

# query
response = query_engine.query("面对全球保护主义，中国电动汽车出口方式有何变化？")
response.print_response_stream()

面对全球保护主义的挑战，中国电动汽车企业在出口策略和方式上出现了显著的变化。主要体现在以下几个方面：

1. **市场多元化**：积极开拓新兴市场，如南亚、东南亚、中东等地区，以分散对单一市场的依赖，并寻找新的增长点。

2. **技术创新与产品优化**：加强技术研发，提升产品质量和性能，开发具有竞争力的新能源汽车产品。通过技术创新来提高能效、减少碳排放，满足不同国家和地区的需求。

3. **本地化生产与合作**：与中国本土汽车制造商进行合作或投资，如大众投资小鹏、Stellantis投资零跑、奥迪与上汽合作等案例，以降低贸易壁垒和关税成本，并适应当地市场规则。

4. **供应链优化**：调整供应链策略，确保关键原材料的供应安全性和稳定性。同时，提高电池材料的本土化率，减少对进口的依赖。

5. **政策应对与合规性**：密切关注国际规则变化，如欧盟《外国补贴条例》等，确保企业经营活动符合相关法规要求，避免潜在的反补贴调查和罚款风险。

6. **品牌建设和市场营销**：加强全球范围内的品牌建设，提高中国电动汽车品牌的知名度和接受度。通过提供优质的客户服务和支持来增强消费者信任，并在新兴市场进行有效的营销活动。

7. **技术创新与生态融合**：推动新能源汽车与信息技术、大数据、人工智能等领域的深度融合，开发智能网联汽车等创新产品，以技术优势应对市场挑战。

这些变化反映了中国电动汽车行业在全球保护主义背景下的适应策略和长期发展计划。通过多元化市场布局、提升产品质量和技术实力、优化供应链管理以及加强政策合规性，中国电动汽车企业旨在在全球竞争中保持竞争力并实现可持续增长。CPU times: user 544 ms, sys: 31.9 ms, total: 576 ms
Wall time: 1min 19s
